In [ ]:
import csv
import random
with open('ClickData4.tsv') as f:
    user=f.readlines()

with open('ClickData4.tsv','w') as f:
    for i in user:
        f.write(i)

In [ ]:
import datetime
import time
def timeconvert(timestr):
    ifpm=False
    if 'PM' in timestr:
        ifpm=True
    tp=datetime.datetime.strptime(timestr[:-3], "%m/%d/%Y %H:%M:%S")
    tp+=datetime.timedelta(hours=12)
    return tp

In [ ]:
import csv
with open('DocMeta3.tsv') as f:
    data=f.readlines()


In [ ]:
import nltk 
from nltk.tokenize import word_tokenize
news={}
category={}
subcategory={}
for i in data:
    tp=i.strip().split('\t')
    news[tp[1]]=[tp[2],tp[3],word_tokenize(tp[6].lower()),word_tokenize(tp[7].lower())]
    category[tp[2]]=0
    subcategory[tp[3]]=0

In [ ]:
category={'None':0}
subcategory={'None':0}
for i in data:
    tp=i.strip().split('\t')
    if tp[2] not in category:
        category[tp[2]]=len(category)
    if tp[3] not in subcategory:
        subcategory[tp[3]]=len(subcategory)

In [ ]:
userid_dict={}
for i in user:
    tr=i.strip().split('\t')
    userid=tr[0]
    if userid not in  userid_dict:
        userid_dict[userid]=len(userid_dict)

In [ ]:
word_dict0={'PADDING':[0,999999]}

for i in news:
    for j in news[i][2]:
        if j in word_dict0:
            word_dict0[j][1]+=1
        else:
            word_dict0[j]=[len(word_dict0),1]
for i in news:
    for j in news[i][3]:
        if j in word_dict0:
            word_dict0[j][1]+=1
        else:
            word_dict0[j]=[len(word_dict0),1]


In [ ]:
word_dict={}
for i in word_dict0:
    if word_dict0[i][1]>=3:
        word_dict[i]=[len(word_dict),word_dict0[i][1]]
print(len(word_dict),len(word_dict0))

In [ ]:
len(news)

In [ ]:
import numpy as np
embdict={}
plo=0
import pickle
with open('/data/wuch/glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        j=f.readline()
        if len(j)==0:
            break
        k = j.split()
        word=k[0].decode()
        linenb+=1
        if len(word) != 0:
            tp=[float(x) for x in k[1:]]
            if word in word_dict:
                embdict[word]=tp
                if plo%100==0:
                    print(plo,linenb,word)
                plo+=1



In [ ]:
from numpy.linalg import cholesky
word_dict1=word_dict
print(len(embdict),len(word_dict1))
print(len(word_dict1))
lister=[0]*len(word_dict1)
xp=np.zeros(300,dtype='float32')

cand=[]
for i in embdict.keys():
    lister[word_dict1[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(lister[word_dict1[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm, 300)
lister[0]=np.zeros(300,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)

In [ ]:
from keras.utils.np_utils import *

In [ ]:
news_words=[[0]*30]
news_index={'0':0}
for i in news:
    tp=[]
    news_index[i]=len(news_index)
    for j in news[i][2]:
        if j in word_dict:
            tp.append(word_dict[j][0])
    tp=tp[:30]
    news_words.append(tp+[0]*(30-len(tp)))

In [ ]:
news_body=[[0]*300]
news_index={'0':0}
for i in news:
    tp=[]
    news_index[i]=len(news_index)
    for j in news[i][3]:
        if j in word_dict:
            tp.append(word_dict[j][0])
    tp=tp[:300]
    news_body.append(tp+[0]*(300-len(tp)))

In [ ]:
news_v=[to_categorical(0,len(category))]
news_sv=[to_categorical(0,len(subcategory))]
for i in news:
    news_v.append(to_categorical(category[news[i][0]],len(category)))
for i in news:
    news_sv.append(to_categorical(subcategory[news[i][1]],len(subcategory)))

In [ ]:
import numpy as np
news_words=np.array(news_words,dtype='int32') 

In [ ]:
news_body=np.array(news_body,dtype='int32') 

In [ ]:
news_v=np.array(news_v,dtype='int32') 
news_sv=np.array(news_sv,dtype='int32') 

In [ ]:
import random
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [ ]:
import itertools
import random
npratio=4
all_train_id=[]
all_train_pn=[]    
all_labeler=[]

all_test_id=[]
all_test_pn=[]    
all_test_labeler=[]
all_test_index=[]

all_user_pos=[]
all_test_user_pos=[]

for raw in user:
    tr=raw.strip().split('\t')
    userid=tr[0]
    if len(tr)==4:
        
        tp=[x.split('#TAB#') for x in tr[2].split('#N#')]
    if len(tr)==3:
        tp=[x.split('#TAB#') for x in tr[2].split('#N#')]

    trainpos=[x[0].split() for x in tp]
    trainneg=[x[1].split() for x in tp]
     
    ppp=list(itertools.chain(*(trainpos)))
    nnn=list(itertools.chain(*(trainneg)))
    #print(ppp)
    
    
    if len(tr)==4:
        tp=[x.split('#TAB#') for x in tr[3].split('#N#')]
        testpos=[x[0].split() for x in tp]
        testneg=[x[1].split() for x in tp]
        
        
        for i in range(len(testpos)):
            tp=[]
            tp.append(len(all_test_pn))
            qqq=list(set(ppp))
            allpos=[news_index[p] for p in random.sample(qqq,min(50,len(qqq)))[:50]]
            allpos+=[0]*(50-len(allpos))
    
            
            for j in testpos[i]:
                all_test_pn.append(news_index[j])
                all_test_labeler.append(1)
                all_test_id.append(userid_dict[userid])
                all_test_user_pos.append(allpos)
                
            for j in testneg[i]:
                all_test_pn.append(news_index[j])
                all_test_labeler.append(0)
                all_test_id.append(userid_dict[userid])
                all_test_user_pos.append(allpos)
            tp.append(len(all_test_pn))
            all_test_index.append(tp)
            
    #nnnnn=list(set(nnn)-set(ppp))

            
    for mp in range(len(trainpos)):
        for ps in trainpos[mp]:
            #tql=list(set(nnn)-set(trainpos[mp]))
            negps=newsample(trainneg[mp],npratio)
            negps.append(ps)
            tplb=[0]*npratio+[1]
            tid=list(range(npratio+1))
            random.shuffle(tid)

            
            yp=[]
            yplb=[]
            for j in tid:
                yp.append(news_index[negps[j]])
                yplb.append(tplb[j])
            
            qqq=list(set(ppp)-set([ps]))
            allpos=[news_index[p] for p in random.sample(qqq,min(50,len(qqq)))[:50]]
            allpos+=[0]*(50-len(allpos))
            all_train_pn.append(yp)
            all_labeler.append(yplb)
            all_train_id.append(userid_dict[userid])
            all_user_pos.append(allpos)


In [ ]:

def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)


In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *


In [ ]:

def generate_batch_data_random2(all_train_pn,all_labeler,all_train_id,batch_size):
    idx = np.arange(len(all_labeler))
    np.random.shuffle(idx)
    y=all_labeler
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            itx = news_words[all_train_pn[i]]
            itx2 = news_body[all_train_pn[i]]
            itx3 = news_v[all_train_pn[i]]
            itx4 = news_sv[all_train_pn[i]]
            tkk=all_user_pos[all_train_id[i]]
            itxx=[itx[:,k,:] for k in range(itx.shape[1])]
            itxx2=[itx2[:,k,:] for k in range(itx2.shape[1])]
            itxx3=[itx3[:,k,:] for k in range(itx3.shape[1])]
            itxx4=[itx4[:,k,:] for k in range(itx4.shape[1])]
            usx=news_words[all_user_pos[i]]
            usxx=[usx[:,k,:] for k in range(usx.shape[1])]
            usx2=news_body[all_user_pos[i]]
            usxx2=[usx2[:,k,:] for k in range(usx2.shape[1])]
            usx3=news_v[all_user_pos[i]]
            usxx3=[usx3[:,k,:] for k in range(usx3.shape[1])]
            usx4=news_sv[all_user_pos[i]]
            usxx4=[usx4[:,k,:] for k in range(usx4.shape[1])]
            yy=all_labeler[i]
            
            yield (itxx +usxx+itxx2 +usxx2+itxx3 +usxx3+itxx4 +usxx4, [yy])

In [ ]:
def generate_batch_data2(all_train_pn,all_labeler,all_train_id,batch_size):
    idx = np.arange(len(all_labeler))
    y=all_labeler
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            itx = news_words[all_train_pn[i]]
            itx2 = news_body[all_train_pn[i]]
            itx3 = news_v[all_train_pn[i]]
            itx4 = news_sv[all_train_pn[i]]
            usx=news_words[all_test_user_pos[i]]
            usxx=[usx[:,k,:] for k in range(usx.shape[1])]
            usx2=news_body[all_test_user_pos[i]]
            usxx2=[usx2[:,k,:] for k in range(usx2.shape[1])]
            usx3=news_v[all_test_user_pos[i]]
            usxx3=[usx3[:,k,:] for k in range(usx3.shape[1])]
            usx4=news_sv[all_test_user_pos[i]]
            usxx4=[usx4[:,k,:] for k in range(usx3.shape[1])]
            yy=all_labeler[i]
            yield ([itx]+ usxx+[itx2]+ usxx2+[itx3]+ usxx3+[itx4]+ usxx4, [yy])

In [ ]:
all_train_pn=np.array(all_train_pn,dtype='int32')
all_labeler=np.array(all_labeler,dtype='int32')
all_train_id=np.array(all_train_id,dtype='int32')
all_test_pn=np.array(all_test_pn,dtype='int32')
all_test_labeler=np.array(all_test_labeler,dtype='int32')
all_test_id=np.array(all_test_id,dtype='int32')
all_user_pos=np.array(all_user_pos,dtype='int32')
all_test_user_pos=np.array(all_test_user_pos,dtype='int32')

In [ ]:
import itertools
import keras
import random
results=[]
keras.backend.clear_session()
for npratio in range(4,5):

    MAX_SENT_LENGTH=30
    MAX_SENTS=50
    
    
    
    
    MAX_SENT_LENGTH2=300
    sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    
    sentence_input2 = Input(shape=(MAX_SENT_LENGTH2,), dtype='int32')
    embedding_layer = Embedding(len(word_dict), 300, weights=[lister],trainable=True)
    
    embedded_sequences = embedding_layer(sentence_input)
    d_et=Dropout(0.2)(embedded_sequences)
    
    embedded_sequences2 = embedding_layer(sentence_input2)
    d_et2=Dropout(0.2)(embedded_sequences2)
    
    l_cnnt = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(d_et)
    d_ct=Dropout(0.2)(l_cnnt)
    
    attention = Dense(200,activation='tanh')(d_ct)
    attention = Flatten()(Dense(1)(attention))
    attention_weight = Activation('softmax')(attention)
    l_attt=keras.layers.Dot((1, 1))([d_ct, attention_weight])
    
    l_cnnt2 = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(d_et2)
    d_ct2=Dropout(0.2)(l_cnnt2)
    
    attention2 = Dense(200,activation='tanh')(d_ct2)
    attention2 = Flatten()(Dense(1)(attention2))
    attention_weight2 = Activation('softmax')(attention2)
    l_attt2=keras.layers.Dot((1, 1))([d_ct2, attention_weight2])

    vinput=Input((1,), dtype='int32') 
    svinput=Input((1,), dtype='int32') 
    v_embedding_layer = Embedding(len(category)+1, 50,trainable=True)
    sv_embedding_layer = Embedding(len(subcategory)+1, 50,trainable=True)
    v_embedding=Dense(400,activation='relu')(Flatten()(v_embedding_layer(vinput)))
    sv_embedding=Dense(400,activation='relu')(Flatten()(sv_embedding_layer(svinput)))

    all_channel=[l_attt,l_attt2,v_embedding,sv_embedding]
        
    candidate_vecssent50=concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(channel) for channel in all_channel],axis=1)
    
    attentionv = Dense(200,activation='tanh')(candidate_vecssent50)
    
    attention_weightv =Lambda(lambda x:K.squeeze(x,axis=-1))(Dense(1)(attentionv))
    attention_weightv =Activation('softmax')(attention_weightv)

    weightv=keras.layers.Dot((1, 1))([candidate_vecssent50, attention_weightv])
    
    sentEncodert = Model([sentence_input,sentence_input2,vinput,svinput],weightv)
    
    review_input = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(MAX_SENTS)]
    
    review_input2 = [keras.Input((MAX_SENT_LENGTH2,), dtype='int32') for _ in range(MAX_SENTS)]
    
    review_input3 = [keras.Input((1,), dtype='int32') for _ in range(MAX_SENTS)]
    
    review_input4 = [keras.Input((1,), dtype='int32') for _ in range(MAX_SENTS)]
    candidate_4vecssent = [sentEncodert([review_input[_],review_input2[_],review_input3[_],review_input4[_] ]) for _ in range(MAX_SENTS)]
    candidate_vecssent5 =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(review) for review in candidate_4vecssent],axis=1)    
    
    attentionn = Dense(200,activation='tanh')(candidate_vecssent5)
    attentionn =Flatten()(Dense(1)(attentionn))
    attention_weightn = Activation('softmax')(attentionn)
    l_att2=keras.layers.Dot((1, 1))([candidate_vecssent5, attention_weightn])
    #l_att2=AttLayer()(candidate_vecssent2)
    
    candidates = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(1+npratio)]
    
    candidates2 = [keras.Input((MAX_SENT_LENGTH2,), dtype='int32') for _ in range(1+npratio)]
    
    candidates3 = [keras.Input((1,), dtype='int32') for _ in range(1+npratio)]
    
    candidates4 = [keras.Input((1,), dtype='int32') for _ in range(1+npratio)]
    candidate_vecs5 = [sentEncodert([candidates[_],user_id,candidates2[_],candidates3[_],candidates4[_]]) for _ in range(1+npratio)]
    
    logits = [keras.layers.dot([l_att2, candidate_vec], axes=-1) for candidate_vec in candidate_vecs5]
    logits = keras.layers.Activation(keras.activations.softmax)(keras.layers.concatenate(logits))

    
    model = Model(candidates+review_input+candidates2+review_input2+candidates3+review_input3+candidates4+review_input4, logits)
    
    
    candidate_one = keras.Input((MAX_SENT_LENGTH,))
    candidate_one_vec = sentEncodert([candidate_one])
    
    candidate_one2 = keras.Input((MAX_SENT_LENGTH2,))
    candidate_one_vec2 = sentEncodert2([candidate_one2])
    
    candidate_one3 = keras.Input((1,))
    candidate_one_vec3 = sentEncodert3([candidate_one3])
    
    candidate_one4 = keras.Input((1,))
    candidate_one_vec4 = sentEncodert4([candidate_one4])
    
    candidate_one_vec5=sentEncodert([candidate_one_vec,candidate_one_vec2,candidate_one_vec3,candidate_one_vec4])
    
    score = keras.layers.Activation(keras.activations.sigmoid)(
        keras.layers.dot([l_att2, candidate_one_vec5], axes=-1))
    model2 = keras.Model([candidate_one,candidate_one2,candidate_one3,candidate_one4]
                         +review_input+review_input2+review_input3+review_input4, score)
    
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

    for ep in range(3):
        traingen=generate_batch_data_random2(all_train_pn,all_labeler,all_train_id, 30)
        
        model.fit_generator(traingen, epochs=1,steps_per_epoch=len(all_train_id)//30)
        #for ep in range(1):    
        valgen=generate_batch_data2(all_test_pn,all_test_labeler,all_test_id, 30)
        cr = model2.predict_generator(valgen, steps=len(all_test_id)//30,verbose=1)
        from sklearn.metrics import roc_auc_score
        all_auc=[]
        all_mrr=[]
        all_ndcg=[]
        all_ndcg2=[]
        for m in all_test_index:
            if np.sum(all_test_labeler[m[0]:m[1]])!=0 and m[1]<len(cr):
        
                all_auc.append(roc_auc_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0]))
                all_mrr.append(mrr_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0]))
                all_ndcg.append(ndcg_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0],k=5))
                all_ndcg2.append(ndcg_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0],k=10))
        results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
        print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))